In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

op = Options()
op.set_preference('javascript.enabled', False)
print("Starting selenium driver")
driver = webdriver.Firefox(options=op)

inicializando driver do selenium


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

def extract_boato(url, category = None):
  try:
    print("get", url)
    #Go to the address
    driver.get(url)
    print("find published")
    #Extracting publication date
    published = driver.find_element(By.CLASS_NAME, "published").text
    print("published:", published)

    if not category:
      category = driver.find_element(By.CLASS_NAME, "cat-links").text
      print("category:", category)

    #Extracting title
    title = driver.find_element(By.CLASS_NAME, "entry-title").text
    print("Title:", title)

    #Extracting texts in red
    red_elements = driver.find_elements(By.CLASS_NAME,"red")

    if not red_elements:
      red_elements = driver.find_elements_by_xpath('//span[@style="color: #ff0000;"]')
    
    if not red_elements:
      red_elements = driver.find_elements(By.TAG_NAME,"blockquote")
      red_elements = [x for x in red_elements if not x.text.startswith('Confira também')]

    if not red_elements:
      red_elements = driver.find_elements(By.TAG_NAME,"i")
      red_elements = [x for x in red_elements if x.text]

    if not red_elements:
      red_elements = driver.find_elements(By.TAG_NAME,"em")
      red_elements = [x for x in red_elements if x.text]
    
    red_texts = []
    for r in red_elements:
      if r.text:
        red_texts.append(r.text)

    #Extracting paragraphs. They are tagged p
    contents = driver.find_elements(By.TAG_NAME,"p")
    content_texts = [content.text for content in contents if content.text != '']

    #The first paragraph is a subtitle
    subtitle = content_texts[0]
    print("Subtitle:", subtitle)

    #Join paragraphs with a \n and replace original news with the text "_NOTICIA_ORIGINAL_"
    content = '\n '.join(content_texts[1:])
    for red_text in red_texts:
      content = content.replace(red_text, '_NOTICIA_ORIGINAL_')
      r = red_text.replace("\n","\n ")
      content = content.replace(r, '_NOTICIA_ORIGINAL_')

    regexp = re.compile(r'_NOTICIA_ORIGINAL_[\s\n]*_NOTICIA_ORIGINAL_')
    while regexp.search(content):
      content = re.sub(r'_NOTICIA_ORIGINAL_[\s\n]*_NOTICIA_ORIGINAL_', '_NOTICIA_ORIGINAL_', content) # ; seguido de maiuscula, que provavelmente indica fim de frase
    
    blocos_noticia = content.count('_NOTICIA_ORIGINAL_')
    if blocos_noticia > 1:
      return {'url': url, 'status':'MAIS_DE_UM_VERMELHO', 'title':title, 'subtitle':'', 'original_news':'', 'content':'', 'published': published, 'category': category}
    elif blocos_noticia == 0:
      return {'url': url, 'status':'SEM_VERMELHO', 'title':title, 'subtitle':'', 'original_news':'', 'content':'', 'published': published, 'category': category}

    return {'url': url,'title': title, 'subtitle': subtitle, 'original_news': '\n'.join(red_texts), 'content': content, 'status':'OK', 'published': published, 'category': category}
  except Exception as e:
    return {'url': url, 'status':'ERRO', 'title':'', 'subtitle':'', 'original_news':'', 'content':str(e), 'published': '', 'category': ''}

## Reading urls

In [4]:
from pathlib import Path
from datetime import datetime
import csv

x = 'bases/fakepedia_urls_without_duplicates.txt'
urls = []
with Path(x).open('r', encoding="utf-8") as f:
  read_urls = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
  for url in read_urls:
    #print(url)
    urls.append(url['url_review'])

## Extracting from boatos.org

In [ ]:
from pathlib import Path
from datetime import datetime
import csv

documents_path = 'bases/fakepedia_boatos.tsv'

last_id = ''
try:
    with Path(documents_path).open('r', encoding="utf-8") as f:
        read_documents = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
        for doc in read_documents:
            last_id = doc['url']
            #print(last_id)
except:
    pass
print('last_id:', last_id)

ignore_claims = (last_id != '')

with Path(documents_path).open('a', encoding="utf-8", newline='') as f:
    fieldnames = ['url', 'title', 'subtitle', 'original_news', 'content', 'status', 'published', 'category', 'row']
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t', extrasaction='ignore')
    if last_id == '':
        writer.writeheader()
    row = 0
    for url in urls:
        if ignore_claims or url == last_id:
            if url == last_id:
                ignore_claims = False
            continue            
        print(str(datetime.now()), row, url)
        boato = extract_boato(url)
        boato['row'] = row
        row += 1
        writer.writerow(boato)
        f.flush()

In [6]:
#If necessary, it is possible to get URLs per page

urls = []
for i in range(1):
  url = f"https://www.boatos.org/category/politica/page/{i+1}"
  driver.get(url)
  list_published = driver.find_elements(By.CLASS_NAME, "entry-title")
  contents = driver.find_elements_by_css_selector(".entry-title a")
  for link in contents:
    urls.append(link.get_property("href"))